In [20]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.layers import LabelSmoothingCrossEntropy, CrossEntropyFlat
from fastai.data_block import CategoryListBase, CategoryProcessor
from fastai.core import  ItemBase

In [22]:
folder = Path('D:/ProgProjekte/Python/Results-Exact-Study/Patches')

In [23]:
path = folder/"ExpertAlgorithm/NrExperts"

In [24]:
class LabelSmoothingVotesCrossEntropy(nn.Module):
    def __init__(self, max_votes:int=10, min_votes:int=0, scale:float=0.95, eps:float=0.05, reduction='mean'):
        super().__init__()
        self.crit = CrossEntropyFlat(reduction="none")
        self.eps,self.reduction = eps,reduction
        self.max_votes, self.min_votes, self.scale = max_votes, min_votes, scale

    def calc_scale(self, vote):
        #return 0.5 + ((((vote - self.min_votes) / (self.max_votes - self.min_votes)) * self.scale) / 2)
        return (((vote - self.min_votes) / (self.max_votes - self.min_votes)) * self.scale)

    def forward(self, output, target, target_votes):
        loss = self.crit(output, target.long())

        lambd = [self.calc_scale(x.float()) for x in target_votes]
        lambd = loss.new(lambd)

        d = loss * lambd

        if self.reduction == 'mean':
            return d.mean()
        elif self.reduction == 'sum':
            return d.sum()
        return d


class CategoryProcessorWithVotes(CategoryProcessor):

    def generate_classes(self, items):
        "Generate classes from `items` by taking the sorted unique values."
        return uniqueify(items[:, 0], sort=True)

    def process_one(self,item):
        if isinstance(item, EmptyLabel): return item
        res = self.c2i.get(item[0],None)
        if res is None: self.warns.append(str(item))
        return [res, item[1]]

class CategoryWithVotes(ItemBase):
    "Basic class for single classification labels."
    def __init__(self,data, obj, votes):
        self.datad, self.obj, self.votes = data, obj, votes
    def __int__(self):  return int(self.datad)
    def __str__(self):  return str(self.obj)
    def __hash__(self): return hash(str(self))

    @property
    def data(self):
        return (self.datad, self.votes)


class MixedCategoryListWithFloat(CategoryListBase):
    "Basic `ItemList` for single classification labels."
    _processor=CategoryProcessorWithVotes
    def __init__(self, items:Iterator, classes:Collection=None, label_delim:str=None, **kwargs):
        super().__init__(items, classes=classes, **kwargs)
        self.loss_func = CrossEntropyFlat()

    def get(self, i):
        o = self.items[i]
        if o is None: return None
        return CategoryWithVotes(o[0], self.classes[o[0]], o[1])

    def analyze_pred(self, pred, thresh:float=0.5): return pred.argmax()

    def reconstruct(self, t):
        return CategoryWithVotes(t[0], self.classes[t[0]], t[1])
    
def accuracy(input:Tensor, targs:Tensor, vote:Tensor)->Rank0Tensor:
    "Computes accuracy with `targs` when `input` is bs * n_classes."
    n = targs.shape[0]
    input = input.argmax(dim=-1).view(n,-1)
    targs = targs.view(n,-1)
    #from IPython.core.debugger import set_trace; set_trace();
    return (input==targs.long()).float().mean()

In [25]:
tfms = get_transforms(do_flip=True, 
                      flip_vert=True, 
                      max_rotate=90,  
                      #max_lighting=0.0, 
                      #max_zoom=2, 
                      #max_warp=0.2,
                      #p_affine=0.75,
                      #p_lighting=0.75,  
                      #xtra_tfms=xtra_tfms,
                     )

In [26]:
def get_float_labels(x: Path, max_votes:int=10):
    #return x.parent.stem
    votes = max_votes
    if '_' in x.stem:
        votes = int(x.stem.split('_')[0])

    return (x.parent.stem, votes)

def get_data(folder, bs, size, max_votes:int=10):
    return (ImageList.from_folder(folder)
        .split_by_folder(train="train", valid="val")
        .label_from_func(partial(get_float_labels, max_votes=max_votes), label_cls=MixedCategoryListWithFloat)
        .transform(tfms, size=size)
        .databunch(bs=bs, num_workers=0)).normalize()

In [27]:
results = {}
for user in path.glob("*"):
    folder = path/user
    print(folder)
    
    acc_values = []
    for i in range(5):
        
        max_votes = int(user.stem.split("_")[1])
        crit = LabelSmoothingVotesCrossEntropy(max_votes=max_votes)
        learn = cnn_learner(get_data(folder, bs=64, size=64, max_votes=max_votes), models.resnet18, metrics=[accuracy], callback_fns=[], loss_func=crit)

        lr = 0.05
        learn.fit_one_cycle(3, slice(lr))
        learn.unfreeze()

        lr = 0.001
        learn.fit_one_cycle(5, slice(lr))
        
        acc_values.append(float(np.max(learn.recorder.metrics)))   
    
    results[user.stem] = np.mean(acc_values)

D:\ProgProjekte\Python\Results-Exact-Study\Patches\ExpertAlgorithm\NrExperts\NrExperts_10


epoch,train_loss,valid_loss,accuracy,time
0,0.611138,0.735141,0.770529,00:19
1,0.485029,0.418238,0.837458,00:20
2,0.376996,0.311186,0.871204,00:24


epoch,train_loss,valid_loss,accuracy,time
0,0.267496,0.300126,0.868391,00:24
1,0.215933,0.286542,0.875703,00:24
2,0.189413,0.236763,0.899325,00:24
3,0.164504,0.224705,0.903262,00:24
4,0.150683,0.223540,0.907762,00:25


epoch,train_loss,valid_loss,accuracy,time
0,0.597937,1.008189,0.749719,00:23
1,0.467832,0.478002,0.831271,00:25
2,0.349059,0.419889,0.815523,00:25


epoch,train_loss,valid_loss,accuracy,time
0,0.228864,0.302581,0.863330,00:25
1,0.194791,0.351913,0.853768,00:26
2,0.165365,0.244828,0.899325,00:27
3,0.146559,0.236218,0.901575,00:28
4,0.139354,0.231645,0.896513,00:27


...

epoch,train_loss,valid_loss,accuracy,time
0,0.232607,0.448876,0.840832,00:27
1,0.192772,0.245865,0.901012,00:26
2,0.168554,0.272296,0.889764,00:27
3,0.148854,0.240588,0.892013,00:27
4,0.136168,0.222971,0.904949,00:26


epoch,train_loss,valid_loss,accuracy,time
0,0.601985,0.835841,0.697975,00:25
1,0.465879,0.512077,0.806524,00:25
2,0.325591,0.376750,0.834646,00:25


epoch,train_loss,valid_loss,accuracy,time
0,0.205170,0.339549,0.853768,00:26
1,0.178485,0.279844,0.886952,00:27
2,0.159451,0.278261,0.884139,00:27
3,0.148128,0.254401,0.895388,00:27
4,0.131705,0.253113,0.894826,00:27


epoch,train_loss,valid_loss,accuracy,time
0,0.613294,1.017086,0.740157,00:26
1,0.469876,0.514899,0.803150,00:25
2,0.347370,0.373889,0.836895,00:25


epoch,train_loss,valid_loss,accuracy,time
0,0.218018,0.357848,0.848706,00:26
1,0.178250,0.258348,0.897638,00:27
2,0.159945,0.258209,0.885264,00:27
3,0.148737,0.235194,0.897075,00:27
4,0.134820,0.221040,0.902700,00:27


In [28]:
import json
with open('VotesLoss_Algorithm.json', 'w') as fp:
    json.dump(results, fp)

In [29]:
results

{'NrExperts_10': 0.9025871634483338,
 'NrExperts_2': 0.8898762583732605,
 'NrExperts_3': 0.8902137160301209,
 'NrExperts_4': 0.8930258631706238,
 'NrExperts_5': 0.8953880667686462,
 'NrExperts_6': 0.8907761454582215,
 'NrExperts_7': 0.9034870505332947,
 'NrExperts_8': 0.9013498187065124,
 'NrExperts_9': 0.8982002139091492}